In [1]:
import subprocess, shutil, copy
from TprParser.TprReader import TprReader, SimSettings
import MDAnalysis as mda
from MDAnalysis.coordinates.core import triclinic_vectors

In [2]:
def Pressure(fname):
    reader = TprReader(fname)
    ref_p = [
        100, 0, 0,
        0, 100, 0,
        0, 0, 100
    ]
    compress = [
        4.5E-5, 0, 0,
        0, 4.5E-5, 0,
        0, 0, 4.5E-5
    ]
    assert len(ref_p) == 9
    assert len(compress) == 9
    reader.set_pressure('No', 'Isotropic', 1.0, ref_p, compress)


In [3]:
def run_cmd(cmd:str):
    ret = subprocess.run(cmd, shell=True)
    if ret.returncode != 0:
        raise Exception('\nError occurred from command: \n\t%s!!!' %cmd)
    
def MD(inittpr:str, nsteps:int = 10):
    reader = TprReader(inittpr)
    coords = reader.get_xvf('X') # get coords from tpr
    natmA = 120
    natmB = 132
    natm = natmA+natmB

    assert natm == coords.shape[0]
    for i in range(nsteps):
        # move two molecules distance of Z axis each 2.0 A
        tempcoords = copy.deepcopy(coords)
        tempcoords[:natmA,     2] += 0.05 * i
        tempcoords[natmA:natm, 2] -= 0.05 * i
        reader.set_xvf('X', tempcoords)
        # rename new.tpr to em_{i}.tpr
        suffix = inittpr.split(".tpr")[0]+"_"+str(i)
        shutil.move("new.tpr", f"{suffix}.tpr")
        run_cmd(f'gmx mdrun -deffnm {suffix} -v')
    print("Finished!")

In [4]:
def Temperature(fname:str, nsteps:int=2):
    tau_t = [1.0, 1.0] # coupling constant
    reader = TprReader(fname)
    for i in range(nsteps):
        ref_t = [100+i*100, 100+i*100] # 100, 200, 300 K
        reader.set_temperature("Vrescale", tau_t, ref_t)
        suffix = fname.split('.tpr')[0]+"_"+str(i)
        shutil.move("new.tpr", f"{suffix}.tpr")
        run_cmd(f'gmx mdrun -deffnm {suffix} -v')


In [5]:
def MDP_Integer(fname, key, val):
    reader = TprReader(fname)
    reader.set_mdp_integer(key, val)

In [6]:
def get_xvf(fname, type):
    reader = TprReader(fname)
    return reader.get_xvf(type)

In [7]:
def UltraSim(tpr:str="md.tpr", upper:int=100, lower:int= -100, period:float=50):
    """ @brief Ultrasonic simalation by Bidirectional pulse pressure 

    Parameters
    ----------
    tpr: str, default md.tpr
        The tpr filename
    upper: int, default 100 bar
        The perssure upper limitation
    lower: int, default -100 bar
        The perssure lower limitation
    period: float, deafault 50.0 ps
        The period of ultrasonic
    """
    assert upper>lower, 'the upper must be larger than lower value'

    # read tpr, 0.5 upper + 0.5 lower
    ntimes = 5
    totTime = period*ntimes
    print(f'Total simulation time: {totTime} ps')
    for i in range(ntimes*2):
        # no suffix
        newTprName = tpr.split('.tpr')[0]+str(i+1)+'.tpr'

        # pressure
        pressure = upper
        if i%2!=0:
            pressure = lower

        ref_p = [
            pressure, 0, 0,
            0, pressure, 0, 
            0, 0, pressure
        ]
        compress = [
            4.5e-5, 0, 0,
            0, 4.5e-5, 0, 
            0, 0, 4.5e-5
        ]

        # get new tpr by set new pressure
        with SimSettings(tpr, newTprName) as reader:
            reader.set_pressure(
                'ParrinelloRahman', 
                'Isotropic', 
                2.0,
                ref_p, 
                compress)

            # the next should set pressure, new coords and velocity
            if i != 0:
                # get previous coords and velocity from md_x.gro
                u = mda.Universe(tpr.split('.tpr')[0]+str(i)+'.gro')
                prevCoords = u.atoms.positions*0.1 # to nm
                prevVelocity = u.atoms.velocities*0.1 # to nm/ps
                prevBox = triclinic_vectors(u.dimensions)*0.1 # nm

                # modify coords, modify velocity and box
                reader.set_xvf('x', prevCoords)
                reader.set_xvf('v', prevVelocity)
                reader.set_xvf('box', prevBox)

        run_cmd(f'gmx mdrun -deffnm {newTprName.split(".tpr")[0]} -v')


In [8]:
if __name__ == '__main__':
    UltraSim(r'C:\Users\liuyujie714\Desktop\waterbox\md.tpr',
             upper=500, 
             lower=-500,
             period=200
             )



Total simulation time: 1000 ps
